In [ ]:
"""

Created on Wed Jan 22 16:41 2025

Prepare initial conditions for ice-shelf cavities

Author: C. Burgard

"""

In [1]:
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm
from cdo import Cdo
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


In [3]:
cdo = Cdo()
print('this is CDO version %s'%(cdo.version()))

this is CDO version 2.1.0


FUNCTIONS

In [4]:
def weighted_mean(data, dims, weights):
    weight_sum = weights.sum(dim=dims) # to avoid dividing by zero
    return (data*weights).sum(dim=dims)/weight_sum.where(weight_sum != 0)

READ IN DATA

In [5]:
inputpath_raw = '/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/raw/'
inputpath_raw2 = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/'
inputpath_interim = '/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/interim/'
inputpath_interim2 = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/'

In [6]:
T_init_0 = xr.open_dataset(inputpath_raw + 'conservative_temperature_WOA13_decav_Reg1L75_clim.nc')
S_init_0 = xr.open_dataset(inputpath_raw + 'absolute_salinity_WOA13_decav_Reg1L75_clim.nc')
#T_init_0 = xr.open_dataset(inputpath_raw + 'eORCA1.4.2_conservative_temperature_WOCE.nc')
#S_init_0 = xr.open_dataset(inputpath_raw + 'eORCA1.4.2_absolute_salinity_WOCE.nc')

In [7]:
ds_isfNEMO= xr.open_dataset(inputpath_interim + 'masks_for_eORCA1_prescribedmeltinopencav.nc')
domain_cfg = xr.open_dataset(inputpath_raw2 + 'eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights/eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights_domain_cfg.nc')

In [8]:
mask_fronts = xr.open_dataset(inputpath_interim2 + 'mask_fronts_eORCA1.nc')

In [9]:
file_Justine = xr.open_dataset(inputpath_raw2 + 'Mask_Iceshelf_IMBIE2_v2.nc')

In [10]:
isfmask_nn = cdo.remapnn(inputpath_raw + 'griddes_latlon_1deg.txt',input=file_Justine['Iceshelf_extrap'], returnArray='Iceshelf_extrap')
isfmask_extrap_1deg = xr.DataArray(data=isfmask_nn, dims=['y','x'])

In [11]:
isfmask_nn = cdo.remapnn(inputpath_raw + 'griddes_latlon_1deg.txt',input=file_Justine['Iceshelf'], returnArray='Iceshelf')
isfmask_1deg = xr.DataArray(data=isfmask_nn, dims=['y','x'])

In [13]:
isfmask_nn_from_NEMO_tolatlon = xr.open_dataset(inputpath_interim + 'test_put_NEMO_on_1deg.nc')

In [ ]:
isfmask_nn_from_NEMO_tolatlon['mask_isf'].plot()

Define the previously closed regions

In [14]:
landsea_mask = pd.read_csv('/home/cburgard/SCRIPTS/ISFinNEMO_Clara/WOA13_mask.csv',delimiter=',',header=1)

In [15]:
landsea_mask_xr = xr.DataArray(data=np.reshape(landsea_mask['Bottom_Standard_Level'], (180,360)), dims=['y','x']).assign_coords({'x': np.concatenate([np.arange(180,360),np.arange(0,180)]), 'y': np.arange(0,180)})#.assign_coords({'x': np.concatenate([np.arange(0.5,179.6),np.arange(-179.5,-0.4)]), 'y': np.arange(-89.5,89.6)})#.assign_coords({'x': np.arange(-179.5,179.6), 'lat': np.arange(-89.5,89.6)})
ls_mask_01 = landsea_mask_xr.sortby(landsea_mask_xr.x) == 1

Identify the regions in front of the ice shelves I want to open and average T and S

In [ ]:
ls_mask_01.plot()

In [16]:
mask_0_1_2 = ls_mask_01.copy()
mask_0_1_2 = mask_0_1_2.where(mask_0_1_2 != 1,3)
mask_0_1_2 = mask_0_1_2.where(mask_0_1_2 != 0,1)

In [17]:
mask_front0 = mask_0_1_2.where(isfmask_1deg > 0,1).copy()

In [ ]:
mask_front.plot()

In [ ]:
isfmask_1deg.plot()

In [ ]:
(mask_front * 0 + isfmask_extrap_1deg).plot()

In [ ]:
ls_mask_01.where(mask_front == 5).plot()

In [18]:
mask_front = mask_front0.copy()
mask_front = mask_front.where((mask_front0.roll(x=-1)-mask_front0)!=2,5)
mask_front = mask_front.where((mask_front0.roll(x=1)-mask_front0)!=2,5)
mask_front = mask_front.where((mask_front0.roll(y=-1)-mask_front0)!=2,5)
mask_front = mask_front.where((mask_front0.roll(y=1)-mask_front0)!=2,5)
# cut out all front points
mask_front = mask_front.where(mask_front==5)

# set the ice shelf number
mask_front = mask_front * 0 + isfmask_extrap_1deg

MAKE THE VERTICAL MEAN PROFILES

In [19]:
cell_area = xr.open_dataset(inputpath_interim + 'gridarea_1deg.nc')

In [20]:
cell_area = cell_area.rename({'lat':'y','lon':'x'}).assign_coords({'x': np.arange(0,360), 'y': np.arange(0,180)})['cell_area']

In [21]:
T_mean_list = []
S_mean_list = []

for ID in tqdm(file_Justine.ID):
    if ((mask_front == ID).sum() > 1) and (ID != 67) and (ID != 125):
        if ID == 66:
            front_mask_kisf = (mask_front == 66) | (mask_front == 67)
        elif ID == 124:
            front_mask_kisf = (mask_front == 124) | (mask_front == 125)
        else:
            front_mask_kisf = (mask_front == ID)
            
        T_mean = weighted_mean(T_init_0['votemper'].where(front_mask_kisf), ['x','y'], cell_area.where(front_mask_kisf))
        S_mean = weighted_mean(S_init_0['vosaline'].where(front_mask_kisf), ['x','y'], cell_area.where(front_mask_kisf))
        
        T_mean_list.append(T_mean.assign_coords({'ID': ID}))
        S_mean_list.append(S_mean.assign_coords({'ID': ID}))

T_mean_all = xr.concat(T_mean_list, dim='ID')
S_mean_all = xr.concat(S_mean_list, dim='ID')

  0%|          | 0/133 [00:00<?, ?it/s]

CHECK THE PROFILES

In [ ]:
ID_open_list = [21,66,117,124,127,128] #,67,125

In [ ]:
f = plt.figure()
f.set_size_inches(8.25, 8.25)

ax={}

leg_hdl = []
alpha_val = 0.2

i = 0

for kisf in tqdm(ID_open_list):

    ax[i] = f.add_subplot(2,3,i+1)
    

    if kisf not in [62,36]:
        for tt in T_mean_all.time_counter:

            T_data0 = T_mean_all.sel(time_counter = tt, ID = kisf)
            ax[i].plot(T_data0,-T_data0.z, color='orange', alpha=alpha_val)



            #leg, = plt.plot(mean_run,-1*file_TS_mean_all.depth, color=colors[n], linewidth=3)
            #plt.fill_betweenx(-1*mean_run.depth, mean_run - std_run, mean_run + std_run, -1*mean_run.depth, alpha=0.2, color=colors[n])
            #leg_hdl.append(leg)


    ax[i].set_title(str(kisf))
    #ax[i].set_xlim(-2,2)

    i = i+1


f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()

In [ ]:
f = plt.figure()
f.set_size_inches(8.25, 8.25)

ax={}

leg_hdl = []
alpha_val = 0.2

i = 0

for kisf in tqdm(ID_open_list):

    ax[i] = f.add_subplot(2,3,i+1)
    

    if kisf not in [62,36]:
        for tt in S_mean_all.time_counter:

            S_data0 = S_mean_all.sel(time_counter = tt, ID = kisf)
            ax[i].plot(S_data0,-S_data0.z, color='orange', alpha=alpha_val)



            #leg, = plt.plot(mean_run,-1*file_TS_mean_all.depth, color=colors[n], linewidth=3)
            #plt.fill_betweenx(-1*mean_run.depth, mean_run - std_run, mean_run + std_run, -1*mean_run.depth, alpha=0.2, color=colors[n])
            #leg_hdl.append(leg)


    ax[i].set_title(str(kisf))
    #ax[i].set_xlim(-2,2)

    i = i+1


f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()

PROPAGATE THEM

In [22]:
new_T0 = T_init_0.copy()
new_S0 = S_init_0.copy()
for ID in tqdm(T_mean_all.ID):
    new_T0 = new_T0.where(isfmask_1deg != ID, T_mean_all.sel(ID=ID))
    new_S0 = new_S0.where(isfmask_1deg != ID, S_mean_all.sel(ID=ID))


  0%|          | 0/71 [00:00<?, ?it/s]

In [23]:
new_T0 = new_T0.where(isfmask_1deg != 67, T_mean_all.sel(ID=66).drop('ID'))
new_T0 = new_T0.where(isfmask_1deg != 125, T_mean_all.sel(ID=124).drop('ID'))
new_T0['lon'] = T_init_0['lon']
new_T0['lat'] = T_init_0['lat']
new_T0['nav_lev'] = T_init_0['nav_lev']


new_S0 = new_S0.where(isfmask_1deg != 67, S_mean_all.sel(ID=66).drop('ID'))
new_S0 = new_S0.where(isfmask_1deg != 125, S_mean_all.sel(ID=124).drop('ID'))
new_S0['lon'] = S_init_0['lon']
new_S0['lat'] = S_init_0['lat']
new_S0['nav_lev'] = S_init_0['nav_lev']

WRITE TO NETCDF

In [32]:
new_T0.drop_vars('ID').to_netcdf(inputpath_interim + 'conservative_temperature_WOA13_decav_Reg1L75_clim_withisfcav.nc', unlimited_dims=['time_counter'])
new_S0.drop_vars('ID').to_netcdf(inputpath_interim + 'absolute_salinity_WOA13_decav_Reg1L75_clim_withisfcav.nc', unlimited_dims=['time_counter'])

In [26]:
T_init_0

<xarray.Dataset>
Dimensions:       (time_counter: 12, z: 75, y: 180, x: 360)
Coordinates:
  * time_counter  (time_counter) float32 1.0 2.0 3.0 4.0 ... 9.0 10.0 11.0 12.0
Dimensions without coordinates: z, y, x
Data variables:
    votemper      (time_counter, z, y, x) float32 -0.1693 -0.295 ... -0.729
    lon           (x) float32 ...
    lat           (y) float32 ...
    nav_lev       (z) float32 ...
Attributes:
    creation_date:  Wed Jun 10 10:34:40 CEST 2015
    script_name:    /home/globc/sanchez/WORK/TEOS10/5_compute_teos10.ncl

In [30]:
new_T0.drop_vars('ID')

<xarray.Dataset>
Dimensions:       (time_counter: 12, z: 75, y: 180, x: 360)
Coordinates:
  * time_counter  (time_counter) float32 1.0 2.0 3.0 4.0 ... 9.0 10.0 11.0 12.0
Dimensions without coordinates: z, y, x
Data variables:
    votemper      (time_counter, z, y, x) float64 -1.462 -1.462 ... -0.729
    lon           (x) float32 ...
    lat           (y) float32 ...
    nav_lev       (z) float32 ...
Attributes:
    creation_date:  Wed Jun 10 10:34:40 CEST 2015
    script_name:    /home/globc/sanchez/WORK/TEOS10/5_compute_teos10.ncl

CHECK WHY THE FILES ARE SO LARGE

In [ ]:
T0 = xr.open_dataset(inputpath_interim + 'conservative_temperature_WOA13_decav_Reg1L75_clim_withisfcav.nc')

In [ ]:
T_init_0 - T0 

In [ ]:
T0['lon'] = T_init_0['lon']

In [ ]:
T0